In [ ]:
# SentimentScope - Social Media Sentiment Analysis
# Install required packages first:
# pip install textblob transformers torch matplotlib seaborn pandas wordcloud

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from transformers import pipeline
from wordcloud import WordCloud
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# ==========================
# 1. SAMPLE DATA GENERATION
# ==========================

def generate_sample_data(n_samples=200):
    """Generate sample social media data for demonstration"""

    # Sample texts with different sentiments
    positive_texts = [
        "This product is amazing! Best purchase ever!",
        "Absolutely love this service, highly recommend!",
        "Outstanding quality and great customer support!",
        "Exceeded my expectations, couldn't be happier!",
        "Fantastic experience, will definitely buy again!"
    ]

    negative_texts = [
        "Terrible product, complete waste of money",
        "Worst customer service I've ever experienced",
        "Very disappointed, not worth the price",
        "Poor quality, broke after one use",
        "Would not recommend, stay away from this"
    ]

    neutral_texts = [
        "The product arrived on time",
        "It works as described in the specifications",
        "Average quality for the price point",
        "Standard features, nothing special",
        "It's okay, meets basic requirements"
    ]

    # Generate random data
    texts = []
    dates = []

    start_date = datetime.now() - timedelta(days=30)

    for i in range(n_samples):
        # Random sentiment distribution
        rand = np.random.random()
        if rand < 0.4:
            text = np.random.choice(positive_texts)
        elif rand < 0.7:
            text = np.random.choice(negative_texts)
        else:
            text = np.random.choice(neutral_texts)

        texts.append(text)
        dates.append(start_date + timedelta(days=np.random.randint(0, 30)))

    df = pd.DataFrame({
        'text': texts,
        'date': dates
    })

    return df.sort_values('date').reset_index(drop=True)

# ==========================
# 2. SENTIMENT ANALYSIS
# ==========================

def analyze_sentiment_textblob(text):
    """Analyze sentiment using TextBlob"""
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity

    if polarity > 0.1:
        return 'Positive', polarity
    elif polarity < -0.1:
        return 'Negative', polarity
    else:
        return 'Neutral', polarity

def analyze_sentiment_transformer(texts, model_name='distilbert-base-uncased-finetuned-sst-2-english'):
    """Analyze sentiment using Hugging Face Transformers"""
    try:
        sentiment_pipeline = pipeline("sentiment-analysis", model=model_name)
        results = sentiment_pipeline(texts)
        return results
    except Exception as e:
        print(f"Transformer model error: {e}")
        return None





In [ ]:

# ==========================
# 3. VISUALIZATION FUNCTIONS
# ==========================

def plot_sentiment_distribution(df):
    """Plot sentiment distribution pie chart"""
    plt.figure(figsize=(10, 6))

    sentiment_counts = df['sentiment'].value_counts()
    colors = ['#2ecc71', '#e74c3c', '#95a5a6']

    plt.subplot(1, 2, 1)
    plt.pie(sentiment_counts.values, labels=sentiment_counts.index,
            autopct='%1.1f%%', colors=colors, startangle=90)
    plt.title('Sentiment Distribution', fontsize=14, fontweight='bold')

    plt.subplot(1, 2, 2)
    sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values, palette=colors)
    plt.title('Sentiment Counts', fontsize=14, fontweight='bold')
    plt.xlabel('Sentiment')
    plt.ylabel('Count')

    plt.tight_layout()
    plt.show()

def plot_sentiment_over_time(df):
    """Plot sentiment trends over time"""
    plt.figure(figsize=(14, 6))

    # Aggregate by date
    df['date_only'] = df['date'].dt.date
    daily_sentiment = df.groupby(['date_only', 'sentiment']).size().unstack(fill_value=0)

    # Plot stacked area chart
    plt.subplot(1, 2, 1)
    daily_sentiment.plot(kind='area', stacked=True,
                         color=['#2ecc71', '#e74c3c', '#95a5a6'], alpha=0.7)
    plt.title('Sentiment Trends Over Time (Stacked)', fontsize=14, fontweight='bold')
    plt.xlabel('Date')
    plt.ylabel('Number of Posts')
    plt.legend(title='Sentiment')
    plt.xticks(rotation=45)

    # Plot line chart for polarity
    plt.subplot(1, 2, 2)
    daily_polarity = df.groupby('date_only')['polarity'].mean()
    plt.plot(daily_polarity.index, daily_polarity.values, marker='o', linewidth=2, color='#3498db')
    plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    plt.title('Average Sentiment Polarity Over Time', fontsize=14, fontweight='bold')
    plt.xlabel('Date')
    plt.ylabel('Polarity Score')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

def plot_polarity_distribution(df):
    """Plot polarity score distribution"""
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.hist(df['polarity'], bins=30, color='#3498db', edgecolor='black', alpha=0.7)
    plt.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Neutral')
    plt.title('Polarity Score Distribution', fontsize=14, fontweight='bold')
    plt.xlabel('Polarity Score')
    plt.ylabel('Frequency')
    plt.legend()

    plt.subplot(1, 2, 2)
    sns.boxplot(x='sentiment', y='polarity', data=df, palette=['#2ecc71', '#e74c3c', '#95a5a6'])
    plt.title('Polarity by Sentiment Category', fontsize=14, fontweight='bold')
    plt.xlabel('Sentiment')
    plt.ylabel('Polarity Score')

    plt.tight_layout()
    plt.show()

def create_word_cloud(df, sentiment_filter=None):
    """Create word cloud for specific sentiment"""
    if sentiment_filter:
        texts = ' '.join(df[df['sentiment'] == sentiment_filter]['text'])
        title = f'{sentiment_filter} Sentiment Word Cloud'
    else:
        texts = ' '.join(df['text'])
        title = 'Overall Word Cloud'

    wordcloud = WordCloud(width=800, height=400,
                          background_color='white',
                          colormap='viridis',
                          max_words=100).generate(texts)

    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title, fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# ==========================
# 4. MAIN ANALYSIS PIPELINE
# ==========================

def run_sentiment_analysis():
    """Main function to run complete sentiment analysis"""

    print("=" * 60)
    print("SENTIMENTSCOPE - SOCIAL MEDIA SENTIMENT ANALYSIS")
    print("=" * 60)

    # Generate sample data
    print("\n📊 Generating sample data...")
    df = generate_sample_data(n_samples=200)
    print(f"✓ Generated {len(df)} sample posts")

    # Perform sentiment analysis using TextBlob
    print("\n🔍 Analyzing sentiment with TextBlob...")
    df['sentiment'], df['polarity'] = zip(*df['text'].apply(analyze_sentiment_textblob))
    print("✓ Sentiment analysis complete")

    # Display statistics
    print("\n" + "=" * 60)
    print("SENTIMENT STATISTICS")
    print("=" * 60)
    print(df['sentiment'].value_counts())
    print(f"\nAverage Polarity: {df['polarity'].mean():.4f}")
    print(f"Polarity Range: [{df['polarity'].min():.4f}, {df['polarity'].max():.4f}]")

    # Display sample results
    print("\n" + "=" * 60)
    print("SAMPLE RESULTS")
    print("=" * 60)
    print(df[['text', 'sentiment', 'polarity']].head(10))

    # Visualizations
    print("\n📈 Generating visualizations...")

    plot_sentiment_distribution(df)
    plot_sentiment_over_time(df)
    plot_polarity_distribution(df)
    create_word_cloud(df)

    # Optional: Create word clouds for each sentiment
    for sentiment in ['Positive', 'Negative']:
        create_word_cloud(df, sentiment_filter=sentiment)

    print("\n✓ Analysis complete!")

    return df

In [ ]:
# ==========================
# 5. OPTIONAL: ANALYZE CUSTOM DATA
# ==========================

def analyze_custom_data(texts_list):
    """Analyze custom list of texts"""
    df = pd.DataFrame({
        'text': texts_list,
        'date': [datetime.now()] * len(texts_list)
    })

    df['sentiment'], df['polarity'] = zip(*df['text'].apply(analyze_sentiment_textblob))

    print("\nCustom Data Analysis Results:")
    print(df[['text', 'sentiment', 'polarity']])

    return df


In [ ]:
# ==========================
# RUN THE ANALYSIS
# ==========================

if __name__ == "__main__":
    # Run main analysis
    results_df = run_sentiment_analysis()

    # Example: Analyze custom texts
    # custom_texts = [
    #     "I absolutely love this product!",
    #     "This is the worst experience ever",
    #     "It's okay, nothing special"
    # ]
    # custom_df = analyze_custom_data(custom_texts)